In [1]:

file = 'f'
f = open(f'{file}.txt')

books, numLibs, days = [int(k) for k in f.readline().split()]
scores = [int(k) for k in f.readline().split()]
Libs = []

for i in range(numLibs):
    info = {}
    tmp = [int(k) for k in f.readline().split()]
    info['index'] = i
    info['numBooks'] = tmp[0] #volatile since we change with set opperation
    info['used'] = False
    info['numDays'] = tmp[1]
    info['booksPerDay'] = tmp[2]

    info['books'] = list(set([int(k) for k in f.readline().split()]))
    info['values'] = [0]*len(info['books'])
    for i, bookNumber in enumerate(info['books']):
        info['values'][i] = scores[bookNumber]

    info['values'] = sorted(info['values'])
    info['opportunityCost'] = [sum(info['values'])]

    i = 0
    first = True
    while i < len(info['values'])-1:
        amount = 0
        if first and len(info['books']) % info['booksPerDay'] != 0:
            limit = len(info['books']) % info['booksPerDay']
            first = False
        else:
            limit =  info['booksPerDay']
            first = False
            
        for j in range(limit):
            amount += info['values'][i+j]
        i += limit

        if info['opportunityCost'][-1] > amount:    
            info['opportunityCost'].append(info['opportunityCost'][-1] - amount)
    
    Libs.append(info)

In [2]:
uniqueBooks = set()


for Lib in Libs:

    Lib['values_indexed'] = sorted([(int(scores[book]), book) for book in Lib['books']], reverse=True)
    
    inter = uniqueBooks.intersection(set(Lib['books']))
    
    if (len(inter) / len(Lib['books']) < 0.7):
        uniqueBooks.union(set(Lib['books']))
    else:
        Lib['used'] = True
    #print(Lib['values_indexed'])


In [3]:
def libraryValue(library, daysleft):
    
    numBooks = min(len(library['books']), daysleft * library['booksPerDay'])
    
    #value = library['opportunityCost'][-min(daysleft, len(library['opportunityCost']))]
    value = library['opportunityCost'][-min(daysleft, len(library['opportunityCost']))] / library['numDays']
    
    #value = Lib['ramil_value']
    
    return numBooks, value


In [4]:
def bestLibrary(daysleft):
    
    maxvalue = -1
    maxindex = -1
    maxbooks = -1
    for library in Libs:
        if not library['used']:
            tempbooks, tempvalue = libraryValue(library, daysleft)
            if (tempvalue > maxvalue) and (daysleft > library['numDays']):
                maxindex = library['index']
                maxvalue = tempvalue
                maxbooks = tempbooks
                
    return maxindex, maxbooks


In [5]:
def pickLibary(daysleft):

    chosen, maxbooks = bestLibrary(daysleft)      #SPECIFIY PICKER FUNCTION HERE
    
    #chosen, maxbooks = randomLibrary(daysleft)
    
    Libs[chosen]['used'] = True
    
    return maxbooks, Libs[chosen]
    # should return tuple:
    #         (signup days, library index, number of books, string of books)

In [6]:
daysleft = days
out = ""
outNumLibs = 0

#line = 1

while (daysleft > 0):
    # daysleft ranges from D to 1
    #line += 1

    maxbooks, picked = pickLibary(daysleft)
    if maxbooks == -1:
        break
    out += str(picked['index']) + " " + str(maxbooks) + "\n" + " ".join(map(str, [each[1] for each in picked['values_indexed'][:maxbooks]])) + "\n"
    daysleft -= picked['numDays']
    outNumLibs += 1
    #print(daysleft)
    # pick next best library
    # pick library
    # subtract singiup period from busyleft
    # update output with chosen librarie's
        

    
with open(f'{file}_out.txt', 'w') as f:
    f.write(str(outNumLibs) + "\n")
    f.write(out)
#print(outNumLibs)
#print(out)